In [ ]:
%matplotlib inline 
import torch 
import torch.nn as nn
import torch.distributions as D
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt, log

from sksparse.cholmod import cholesky
import scipy.sparse as sp

from pyvi.utils import GMMLossFunc as lf
from pyvi.utils import HelperFunc as hf


device = ("cuda" if torch.cuda.is_available()
          else "mps" if torch.backends.mps.is_available()
          else "cpu")

print(f"Using {device} device")

plt.style.use('fivethirtyeight')
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "Helvetica"
})


##  Gaussian Process with Matérn $\nu = 1/2$ Covariance Function

### 1. Matérn kernel

The Matérn covariance function is given by
$$c_{\nu}(r; \sigma, \rho) = \sigma^2 \frac{2^{1-\nu}}{\Gamma(\nu)} \left( \sqrt{8\nu} \frac{r}{\rho} \right)^\nu K_{\nu}\left( \sqrt{8\nu} \frac{r}{\rho} \right)$$

When $\nu = 1/2$, this simplifies to:
$$c_{1/2}(r; \sigma, \rho) = \sigma^2 \exp\left( - \frac{2r}{\rho} \right)$$

We use the following parameterization from Fuglstad et al. (2018):

$$ \kappa = \sqrt{8\nu} / \rho \qquad \text{and} \qquad \tau = \sigma \kappa^\nu \sqrt{\frac{\Gamma(\nu + d/2)(4\pi)^{d/2}}{\Gamma(\nu)}},$$
where $d$ is the dimension of the state-space.

For fixed $\nu$, the reciprocal is given by:

$$ \rho =  \sqrt{8\nu} / \kappa  \qquad \text{and} \qquad \sigma =  \tau \kappa^{-\nu} \sqrt{\frac{\Gamma(\nu)}{\Gamma(\nu + d/2)(4\pi)^{d/2}}}$$

When $d = 1$ and $\nu =1/2$,   $\sqrt{\frac{\Gamma(\nu + d/2)(4\pi)^{d/2}}{\Gamma(\nu)}} = \sqrt{2}$. It follows that $$\kappa = 2/ \rho \quad \& \quad\tau = \sigma \sqrt{2\kappa} $$

### 2. PC Prior


* The PC prior for $\tau$ with base model $\tau =0$ is $$\tau|\kappa \sim \text{Exp}(\lambda),$$ with $$\lambda(\kappa) = - \kappa^{-\nu} \sqrt{\frac{\Gamma(\nu)}{\Gamma(\nu + d/2)(4\pi)^{d/2}}} \frac{\log(\alpha)}{\sigma_0}$$ so that $P(\sigma > \sigma_0|\kappa) = \alpha$. 
For $d = 1$ and $\nu =1/2$, $$\lambda(\kappa) = - \frac{1}{\sqrt{2\kappa}}\frac{\log(\alpha)}{\sigma_0}$$

* The PC prior for $\kappa$ with base model $\kappa=0$ is $$\kappa \sim \text{Weibull}(\lambda^{-2/d}, d/2),$$ which satifies $P(\rho <\rho_0) = \alpha$ if $$ \lambda = - \left(\frac{\rho_0}{\sqrt{8\nu}} \right)^{d/2} \log(\alpha).$$ For  $d = 1$ and $\nu =1/2$, this simplifies to 
    $$\kappa \sim \text{Weibull}(\lambda^{-2}, 1/2), \qquad \& \qquad \lambda = - \sqrt{\frac{\rho_0}{2}} \log(\alpha)$$

In [ ]:
#============================================================================================
# define the Weibul-Exponential PC prior for the GMRF hyperparameters
#============================================================================================

class PriorPC():
    '''
        Defines Penalized Complexity (PC) prior class over the hyperparameters `sigma` and `rho` of a GMRF with Matern covariance with smoothness nu=1/2.

        The PC prior is derived using the reparameterization `kappa = 2/rho` and `tau = sigma * sqrt{2 kappa}`. We use this parameterization
        and transform back samples to the `sigma-rho` parameterization.  

        The PC prior sets an exponential distribution on `tau` conditionnal on `kappa` and a Weibul distribution on `kappa`.
            y ~ N(mu, sigma2)
            where both `mu` and `sigma2` are unknown 

        For details about the derivation of the PC prior and a general formulation for any value of the smoothness parameter `nu`, see

        Geir-Arne Fuglstad, Daniel Simpson, Finn Lindgren & Håvard Rue (2019) Constructing Priors that Penalize the Complexity of Gaussian Random Fields,
          Journal of the American Statistical Association, 114:525, 445-452, DOI: 10.1080/01621459.2017.1415907

        The paper can be accessed here: https://doi.org/10.1080/01621459.2017.1415907
    '''
    def __init__(self, rho_0, sigma_0, alpha_rho=0.05, alpha_sigma=0.05):
        # recode hyperparameters. This allows for the object to be updated with no need of re-instantiation
        self.params = {'rho_0':rho_0, 'sigma_0':sigma_0, 'alpha_rho':alpha_rho, 'alpha_sigma':alpha_sigma}
        
        # define the Weibull distribution on `kappa`
        lambda_ = -sqrt(0.5 * self.params['rho_0']) * log(self.params['alpha_rho'])
        self.kappa = D.Weibull(1 / lambda_**2, torch.tensor([0.5]))
        
        # define the conditional exponential distribution for `tau` given  `kappa`
        self.tau = lambda kappa: D.Exponential(- torch.log(torch.tensor([self.params['alpha_sigma']])) / (torch.sqrt(2 * kappa) * self.params['sigma_0']))
        
    def sample(self, shape: torch.Size = ()):
        '''
        Method for sampling `(rho, sigma)` from the PC prior distribution. This is 3-step approach:
            (1) sample: kappa ~ Weibull(lambda, 1/2)
            (2) sample: tau|kappa ~ Exp(lambda(kappa))
            (3) map (kappa, tau) ---> (rho, sigma)

        '''
        kappa = self.kappa.sample(shape).squeeze()                # (1) sample: kappa ~ Weibull(lambda, 1/2)
        #kappa = torch.ones(shape) * 2.0
        tau = self.tau(kappa).sample()                            # (2) sample: tau|kappa ~ Exp(lambda(kappa))

        # (3) map (kappa, tau) ---> (rho, sigma)
        rho = 2 / kappa
        sigma = tau / torch.sqrt(2 * kappa)
 
        theta = torch.stack([rho, sigma], dim=-1)       # combine
        return theta.squeeze(), kappa, tau
        
    

In [ ]:
prior = PriorPC(rho_0=.1, sigma_0=10.0, alpha_rho=0.05, alpha_sigma=0.05)
theta, kappa, tau =  prior.sample((10000,))

fig = plt.figure(figsize=(12, 3), dpi=200)

fig.add_subplot(141)
plt.hist(kappa.log(), bins=25, density=True)
plt.title(r'$\log(\kappa)$')

fig.add_subplot(142)
plt.hist(tau.log(), bins=25, density=True)
plt.title(r'$\log(\tau) $')

fig.add_subplot(143)
plt.hist(theta[...,0].log(), bins=25, density=True)
plt.title(r'$\log(\rho)$')

fig.add_subplot(144)
plt.hist(theta[...,1].log(), bins=25, density=True)
plt.title(r'$\log(\sigma)$')

plt.tight_layout()

plt.show()

### 3. Gaussian Process with Matérn Covariance function

In [ ]:
def MaternGP(theta, r, n, n_sample=1):
    '''
        Input:
            - theta = [rho, sigma2]  (tensor), hyperparameters of the covariance function
            - r: resolution, distance between 2 successive points on the grid
            - n: number of points on the grid (regular)
            - n_sample: number of samples to draw from the GP
    '''
    rho, sigma  = theta.tolist()
    sigma2 = sigma ** 2
    #########################################
    # define the Matern precision matrix
    ########################################
    scale = np.exp(- 2 * r / rho) / (sigma2**2 * (1 - np.exp(- 4 * r / rho)))      # scaling factor
    off_diag = - np.ones(n)                                                       # off diagonals elements
    diag = [np.exp(2 * r / rho)] + [np.exp(2 * r / rho) + np.exp(-2 * r / rho)] * (n-2) + [np.exp(2 * r / rho)]   # diagonal elements
    
    data = scale * np.array([off_diag, diag, off_diag])       # put off diagonal and diagonal elements in the same array
    offsets = np.array([-1, 0, 1])        

    Q = sp.dia_matrix((data, offsets), shape=(n,n)).tocsc()          # define the precision matrix as a sparse matrix

    #########################################
    # sampling from the GP
    ########################################
    # compute Cholesky factorization
    factor = cholesky(Q)
    
    # sample z ~ N(0, I)
    z = np.random.randn(n, n_sample)

    # solve L.T @ v = z
    v = factor.solve_Lt(z)

    return v

In [ ]:
def simulator(theta, r, n, n_sample=1):
    Y = np.zeros((theta.shape[0], n, n_sample))

    for i in range(theta.shape[0]):
        y = MaternGP(theta[i], r, n, n_sample)
        Y[i] = y 
    
    Y = torch.from_numpy(Y).to(torch.float32)
    
    return Y.squeeze()

In [ ]:
Y = simulator(theta[:10,:], r=0.1, n=1001, n_sample=1)

In [ ]:
#y = MaternGP(torch.tensor([100.0, 1.0]), 0.01, 10000, 1)

k = np.random.randint(Y.shape[0])
text = r'$\rho = $' + str(round(theta[k].numpy()[0], 1)) + ',  ' +  r'$\sigma = $' + str(round(theta[k].numpy()[1], 1))
y = Y[k]
x = np.linspace(0.0, 100.1, 1001, endpoint=False)

plt.plot(x, y, lw=1.0)
plt.title(text)
plt.show()

### 4. Neural Net

In [ ]:
class Net(nn.Module):
     '''
        Multivariate Gaussian Mixture Density Network class for amortized variational inference

        Gaussian mixtures are dense in the space of probability distributions. This motivates their use for posterior density approximation.

        Each mixture is parameterized by the means, Cholesky factors of associated precision matrices and mixture weights.

        The neural network does not output the Cholesky factors directly but rather tensors containing their respective lower diagonal elements.
    '''
     def __init__(self, input_size, dim:int=2, K:int=1, hd:int=64):
        '''
            Input: 
                * input_size: dimension of the input to the neural net, i.e. the number of elements in the observation vector yobs
                * dim:        dimension of the posterior distribution. This is in general the number of parameters in the model.
                * K:          number of mixture components
                * hd:     dimension of hidden layers
            Output:
                * mean: tensor of dimensions batchsize X k X dim containing the predicted means
                * chol: tensor of appropriate dimensions containing the the predicted Cholesky factors
                * coeff: tensor of appropriate dimensions containing the predicted mixture component weights
        '''
        super().__init__()

        self.dim = dim
        self.K = K
        self.hd = hd
        self.input_size = input_size 


        # convolutional layers
        self.conv = nn.Sequential(
            nn.Conv1d(1, 1, 5),
            nn.ELU(),
            nn.MaxPool1d(3),
            nn.LayerNorm((self.input_size - 4) // 3),
            nn.Conv1d(1, 1, 5),
            nn.ELU(),
            nn.MaxPool1d(2),
            nn.LayerNorm((((self.input_size - 4) // 3) - 4) // 2)
        )

        # fully connected (((self.input_size - 4) // 3) - 4) // 2layers
        self.fc = nn.Sequential(
            nn.Linear((((self.input_size - 4) // 3) - 4) // 2, self.hd),
            nn.ELU(),
            nn.Linear(self.hd, self.hd),
            nn.ELU()
        )
        
        # means of each component
        self.mean = nn.Linear(self.hd, self.K * self.dim)
        
        # Cholesky factors of the precision matrices (diagonal elements are log-scaled)
        self.chol = nn.Linear(self.hd, self.K * self.dim * (self.dim + 1) // 2)

        # mixture weights, non-negative and summing to 1
        self.coeff = nn.Sequential(
            nn.Linear(self.hd, self.K),
            nn.Softmax(dim=1)   
            )  

        
     def forward(self, x):
        # apply convolutional layers
        x = self.conv(x)
        # flatten
        x = x.flatten(1)
        # apply fully-connected layers
        x = self.fc(x)

        # mean
        mean = self.mean(x)
        mean = mean.reshape((mean.shape[0], self.K, self.dim))

        # Cholecky factor
        chol = self.chol(x)
        chol = chol.reshape((chol.shape[0], self.K, self.dim * (self.dim + 1) // 2))

        # mixture weights
        if self.K > 1:
            coeff = self.coeff(x)
        else:
            coeff = torch.ones(1)
        
        return mean, chol, coeff
    
        

### 5. Training

In [ ]:
#Explicitly provided seeds for training process
random_seed = 12345
torch.manual_seed(random_seed)

cuda = torch.cuda.is_available()
if cuda:
    torch.cuda.manual_seed(random_seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)

#========================================== 
#           generating training data
#==========================================
n_prior, n_obs, n_sample, r, batchsize = 10000, 100, 1, 0.1, 100

# sample parameter values from the specified prior
prior = PriorPC(rho_0=2.0, sigma_0=10.0, alpha_rho=0.5, alpha_sigma=0.05)
Theta_train, kappa, tau =  prior.sample((n_prior,))

# draw samples from the simulator conditioned on parameter values
Y_train = simulator(Theta_train, r=r, n=n_obs, n_sample=n_sample)

# put sigma and rho on log-scale for improved training
Theta_train = Theta_train.log()

# create a combined dataset and data loader
data_train = torch.utils.data.TensorDataset(Y_train.unsqueeze(1), Theta_train)
data_loader = torch.utils.data.dataloader.DataLoader(data_train, batch_size=batchsize, shuffle=True)


#==================================================
#  instantiate the Gaussian mixture net
#=================================================
#gmmnet = Net(input_size=n_obs, dim=2, K=2, hd=64).to(device)
gmmnet = hf.MultivariateGaussianMDN(input_size=n_obs, dim=2, K=2, hd=128, sort=True).to(device)

loss_fn = lf.GaussianMixtureLoss(aggr='mean')

# # train DNN model
gmmnet = hf.nn_optimizer(model=gmmnet, 
                       data_loader=data_loader,
                       loss_fn=loss_fn,
                       learning_rate=1e-4,
                       eps=0.01, 
                       max_epochs=100,
                       verbose=True,
                       echo_after=1,
                       path='trained_models/gmrf1d/gp_fc_notsorted.pt'
                      )

#gmmnet = torch.load('trained_models/gmrf1d/gp_rho_sigma_3.pt')

## Simulation-based calibration check

In [ ]:
def sbc_gaussian(gmmnet, proposal, n_sim = 1e+4, ecdf=True, ecdf_diff=False, logscale=None):
    '''
    Perform simulation-based calibration check for a Gaussian mixture network for posterior approximation

    Input:
        -- gmmnet: Gaussian mixture network, with input size given by `sample_size`
        -- proposal: proposal distribution `theta ~ p(theta)`, usually the same as the prior/proposal distribution used for training
                     Note: must have a `sample()` method
        
        -- generator: function that takes parameter values `theta` as input and generate the corresponding simulator model
                       `x ~ p(x|theta)`  as an instance of a class with a `sample` method
        
        -- sample_size: number of iid samples from `x ~ p(x|theta)` for each values of theta

        -- n_sim: number of simulation from the joint distribution: theta ~ p(theta); x ~ p(x|theta)

        -- ecdf: whether to output an eCDF or a histogram plot, default: ecdf=True

        -- ecdf_diff: whether on the y-axis are the `ecdf(w)` values (if True) or `ecdf(w) - w` values (if False).
                        This is ignored if ecdf=False.
        
        -- logscale: (iterable) contains dimensions of the model parameter vector `theta` that are on log-scale
                        note: we use the standard Python counting, starting at 0

    Note: 95% confidence intervals are based on the  Dvoretzky–Kiefer–Wolfowitz inequality (see https://en.wikipedia.org/wiki/Empirical_distribution_function, accessed: 20-05-2024)
    
    Output: SBC plot as a Pyplot figure
    '''

    # draw samples from the prior/proposal  theta ~ p(theta)
    Theta = proposal.sample((n_sim,))[0]

    # draw samples from the model x ~ p(x|theta)
    X = simulator(Theta, r=r, n=n_obs, n_sample=n_sample).unsqueeze(1)
    
    # ensure all dimensions are on the right scale
    if logscale:
        for i in logscale:
            Theta[:,i] = Theta[:,i].log()     # put sigma2 on logscale

    
    # run the gmmnet
    with torch.no_grad():
        mean, chol, coeff = gmmnet(X.to(device))

    n_component, dim = mean.shape[1], mean.shape[2]

    # calculate Cholesky factors
    chol = torch.vmap(lf.exp_diagonal)(lf.to_triangular(chol.view(n_sim * n_component,  dim * (dim + 1) // 2), dim)).view(n_sim, n_component, dim, dim)

    # caclulate precision matrices
    precision = chol @ chol.transpose(2, 3)

    # calculate covariance matrices
    covariance = torch.linalg.inv(precision) 

    # define GMM variational marginal distributions and calculate cdf values for the true parameter values
    W = torch.zeros((n_sim, dim))   # tensor of cdf values
    
    for j in range(dim):
        # mixture weights
        mix = D.Categorical(coeff)
        # mixture components
        comp = D.Normal(mean[:,:,j], torch.sqrt(covariance[:,:,j,j]))
        # define the mixture
        gmm = D.MixtureSameFamily(mix, comp)
        # evaluate cdf
        W[:,j] = gmm.cdf(Theta[:,j].to(device))


    if ecdf:
        #=====================================================
        # ECDF plot
        #=====================================================
        fig = plt.figure(figsize=(8, 3), dpi=200)

        # Calculate the empirical cumulative distribution function (ECDF)
        eCDF = torch.arange(1, n_sim + 1) / n_sim

        # calculate 95% confidence intervals for the eCDF
        eps = np.sqrt(np.log(2 / 0.05) / (2 * n_sim))
        eCDF_lower, eCDF_upper = eCDF - eps, eCDF + eps

        # exact cdf
        x = np.linspace(0, 1, 100)

        # eCDF for mu
        #===============
        fig.add_subplot(121)

        w = W[:,0].sort().values
        if not ecdf_diff:
            # plot eCDF and true CDF values
            plt.step(w, eCDF, lw=1)
            plt.plot(x, x, 'k--', lw=1)

            # plot 95% confidence bands
            plt.fill_between(w, eCDF_lower, eCDF_upper, color='red', alpha=0.2)

            plt.ylabel(r'$F_{\omega}$')
        else:
            plt.step(w, eCDF - w, lw=1)
            #plt.fill_between(w, eCDF_lower - w, eCDF_upper - w, color='red', alpha=0.1)
            plt.ylabel(r'$F_{\omega} - \omega$')

        plt.xlabel(r'$\omega$')
        plt.title(r'$\log(\rho)$')

        # eCDF plot for sigma2
        #======================
        fig.add_subplot(122)

        w = W[:,1].sort().values
        if not ecdf_diff:
            plt.step(w, eCDF, lw=1)
            plt.plot(x, x, 'k--', lw=1)
            # plot 95% confidence bands
            plt.fill_between(w, eCDF_lower, eCDF_upper, color='red', alpha=0.2)

            plt.ylabel(r'$F_{\omega}$')
        else:
            plt.step(w, eCDF - w, lw=1)
            # plot 95% confidence bands
            #plt.fill_between(w, eCDF_lower - w, eCDF_upper - w, color='red', alpha=0.1)
            plt.ylabel(r'$F_{\omega} - \omega$')

        plt.xlabel(r'$\omega$')
        plt.title(r'$\log(\sigma)$')

        plt.tight_layout()
    else:
        #========================================
        # plot histograms
        #========================================
        fig = plt.figure(figsize=(8, 3), dpi=200)

        # mu
        fig.add_subplot(1,2,1)
        plt.hist(W[...,0], bins=20, density=True, alpha=.6, label=r'$F^{-1}(\mu)$')
        plt.title(r'$\log(\rho)$')
        plt.legend(fontsize=7,  markerscale=.5)

        # log(sigma2)
        fig.add_subplot(1,2,2)
        plt.hist(W[...,1], bins=20, density=True, alpha=.6, label=r'$F^{-1}(\log(\sigma^2))$')
        #plt.title(r'$\sigma^2$')
        plt.legend(fontsize=7,  markerscale=.5)

        plt.title(r'$\log(\sigma)$')
        plt.legend(fontsize=7,  markerscale=.5)

    
    return fig

In [ ]:

f = sbc_gaussian(gmmnet, prior, 1000, ecdf=True, ecdf_diff=False, logscale=[0,1])

posterior mean (+ credible interval) vs true value scatter plot

## Approximate posterior

In [ ]:
theta = prior.sample((2,))[0]

y = simulator(theta, r=r, n=n_obs, n_sample=n_sample).unsqueeze(1)

# run the gmmnet
with torch.no_grad():
    mean, chol, coeff = gmmnet(y.to(device))

n_sim, n_component, dim = mean.shape[0], mean.shape[1], mean.shape[2]

# calculate Cholesky factors
chol = torch.vmap(lf.exp_diagonal)(lf.to_triangular(chol.view(n_sim * n_component,  dim * (dim + 1) // 2), dim)).view(n_sim, n_component, dim, dim)

# # caclulate precision matrices
precision = chol @ chol.transpose(2, 3)

# # calculate covariance matrices
covariance = torch.linalg.inv(precision) 

# mixture weights
mix = D.Categorical(coeff)
# mixture components
comp = D.MultivariateNormal(loc=mean, covariance_matrix=covariance)
# define the mixture
gmm = D.MixtureSameFamily(mix, comp)

# draw samples from the approximate posterior
N = 10000
theta_post = gmm.sample((N,)).cpu().squeeze()

# plotting
fig = plt.figure(figsize=(12, 3), dpi=200)

fig.add_subplot(141)
plt.hist(theta_post[:,0,0], bins=100, density=True)
plt.vlines(x=theta[0][0].log(), ymin=0, ymax=plt.axis()[-1], linestyles='dashed', colors='black', lw=1)
plt.title(r'$\log(\rho)$')

fig.add_subplot(142)
plt.hist(theta_post[:,0,1], bins=100, density=True)
plt.vlines(x=theta[0][1].log(), ymin=0, ymax=plt.axis()[-1], linestyles='dashed', colors='black', lw=1)
plt.title(r'$\log(\sigma) $')

fig.add_subplot(143)
plt.hist(theta_post[:,1,0], bins=100, density=True)
plt.vlines(x=theta[1][0].log(), ymin=0, ymax=plt.axis()[-1], linestyles='dashed', colors='black', lw=1)
plt.title(r'$\log(\rho)$')

fig.add_subplot(144)
plt.hist(theta_post[:,1,1], bins=100, density=True)
plt.vlines(x=theta[1][1].log(), ymin=0, ymax=plt.axis()[-1], linestyles='dashed', colors='black', lw=1)
plt.title(r'$\log(\sigma) $')

plt.suptitle('Gaussian mixture posterior')
plt.tight_layout()
plt.show()

In [ ]:
theta = prior.sample((2,))[0]

y = simulator(theta, r=r, n=n_obs, n_sample=n_sample).unsqueeze(1)

# run the gmmnet
with torch.no_grad():
    mean, chol, coeff = gmmnet(y.to(device))

n_sim, n_component, dim = mean.shape[0], mean.shape[1], mean.shape[2]

# calculate Cholesky factors
chol = torch.vmap(lf.exp_diagonal)(lf.to_triangular(chol.view(n_sim * n_component,  dim * (dim + 1) // 2), dim)).view(n_sim, n_component, dim, dim)

# # caclulate precision matrices
precision = chol @ chol.transpose(2, 3)

# # calculate covariance matrices
covariance = torch.linalg.inv(precision) 

# mixture weights
mix = D.Categorical(coeff)
# mixture components
comp = D.MultivariateNormal(loc=mean, covariance_matrix=covariance)
# define the mixture
gmm = D.MixtureSameFamily(mix, comp)

# draw samples from the approximate posterior
N = 10000
theta_post = gmm.sample((N,)).cpu().squeeze()

# plotting
fig = plt.figure(figsize=(12, 3), dpi=200)

fig.add_subplot(141)
plt.hist(theta_post[:,0,0], bins=25, density=True)
plt.vlines(x=theta[0][0].log(), ymin=0, ymax=plt.axis()[-1], linestyles='dashed', colors='black', lw=1)
plt.title(r'$\log(\rho)$')

fig.add_subplot(142)
plt.hist(theta_post[:,0,1], bins=25, density=True)
plt.vlines(x=theta[0][1].log(), ymin=0, ymax=plt.axis()[-1], linestyles='dashed', colors='black', lw=1)
plt.title(r'$\log(\sigma) $')

fig.add_subplot(143)
plt.hist(theta_post[:,1,0], bins=25, density=True)
plt.vlines(x=theta[1][0].log(), ymin=0, ymax=plt.axis()[-1], linestyles='dashed', colors='black', lw=1)
plt.title(r'$\log(\rho)$')

fig.add_subplot(144)
plt.hist(theta_post[:,1,1], bins=25, density=True)
plt.vlines(x=theta[1][1].log(), ymin=0, ymax=plt.axis()[-1], linestyles='dashed', colors='black', lw=1)
plt.title(r'$\log(\sigma) $')

plt.suptitle('Gaussian mixture posterior')
plt.tight_layout()
plt.show()